In [22]:
import numpy as np
import pandas as pd
data = pd.read_csv("MINT_1522_years.csv")
dt=data
data.drop(data.columns[[0]],axis=1,inplace=True)
window_size = int(len(data)/(8))
splittrain = []
splittest = [] 
#15-19 18-22
for i in range(0, len(data)-4*window_size,window_size):
    splittrain.append(data[i:i+4*window_size])
    splittest.append(data[i+4*window_size:i+5*window_size])

In [21]:
splittest[4]

IndexError: list index out of range

In [23]:
from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
import sympy as sp
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')
td=sp.symbols('td')

In [24]:
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10
nlc = NonlinearConstraint(const, 1 ,1)
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint, Bounds

bounds = [[0, 1], [0, 1],[0, 1], [0, 1],[0, 1],[0, 1],[0, 1], [0, 1],[0, 1],[0,1],[0.125,0.3]]

In [25]:
coll,buy_idx,sell_idx=[],[],[]
count1,count2,count3=0,0,0
def func(p):
    global S,B,resl,resfl
    S=0
    B=0
    resl=[]
    resfl=[]
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    mon=100000
    amount=0
    for i in range(0,len(dfo)):
        r = ((w1*sl1[i]+w2*sl2[i]+w3*sl3[i]+w4*sl4[i]+w5*sl5[i]+w6*sl6[i]+w7*sl7[i]+w8*sl8[i]+w9*sl9[i]+w10*sl10[i])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10))#w1=sp.symbols('w1')
        if i!=0:
            if dfo['adj_close_price'][i]<dfo['adj_close_price'][i-1]*0.8:
                mon+=amount*dfo['adj_close_price'][i]*0.998
                amount=0
        if r>td:
            sumall=mon//dfo['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*dfo['adj_close_price'][i]*1.002
            B+=1
        elif r<-td:
            mon+=amount*dfo['adj_close_price'][i]*0.998
            amount=0
            S+=1
        else:
            pass
    ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
    return -((ret-100000)/1000)
def fobj():
    global sl1,sl2,sl3,sl4,sl5,sl6,sl7,sl8,sl9,sl10,dfo,decision_d_l,bh_test_l,bh_train_l,train_l,test_l
    bh_test_l,bh_train_l,train_l,test_l=[],[],[],[]
    for j in range(len(splittrain)):
        dfo=splittrain[j]
        dfo=dfo.reset_index()
        sl1=dfo['volume_prof_signal']
        sl2=dfo['macd_crossover_signal']
        sl3=dfo['psar_adx_signal']
        sl4=dfo['renko_adx_signal']
        sl5=dfo['aroon_kp_signal']
        sl6=dfo['macd_overzero_signal']
        sl7=dfo['ma5_10_signal']
        sl8=dfo['cdc_signal']
        sl9=dfo['bb_rsi_signal']
        sl10=dfo['rsi_adx_signal']
        result = differential_evolution(func, bounds,constraints=(nlc),maxiter=1500,popsize=80,recombination=0.8) 
        coll.append([result.x,-result.fun,j])
        print([result.x,-result.fun,j])
        train_l.append(-result.fun)
        a=result.x
        b=j
        td=a[10]
        w=a[0:10]
        df=splittest[b]
        decision_d_l=[]
        il=[]
        global count1,count2,count3,sell_idx,buy_idx
        mon=100000
        amount=0
        sumall=0
        df=df.reset_index()
        sl1=dfo['volume_prof_signal']
        sl2=dfo['macd_crossover_signal']
        sl3=dfo['psar_adx_signal']
        sl4=dfo['renko_adx_signal']
        sl5=dfo['aroon_kp_signal']
        sl6=dfo['macd_overzero_signal']
        sl7=dfo['ma5_10_signal']
        sl8=dfo['cdc_signal']
        sl9=dfo['bb_rsi_signal']
        sl10=dfo['rsi_adx_signal']

        for i in range(0,len(df)):
            decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
            if i!=0:
                if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                        mon+=amount*df['adj_close_price'][i]*0.998
                        amount=0
                        sell_idx.append(i)
            if decision_d>td:
                sumall=mon//df['adj_close_price'][i]
                amount+=sumall
                mon-=sumall*df['adj_close_price'][i]*1.002
                count1+=1
                buy_idx.append(i)
            elif decision_d<-td:
                mon+=amount*df['adj_close_price'][i]*0.998
                amount=0
                count2+=1
                sell_idx.append(i)
            else:
                count3+=1
            decision_d_l.append(decision_d)
        ret=mon+amount*df['adj_close_price'][len(df)-1]
        stocknum=100000/dfo['adj_close_price'][0]
        bh_test_l.append(((stocknum*(0.998*df['adj_close_price'][len(df)-1])+(100000-stocknum*df['adj_close_price'][0]*1.002))-100000)/1000)
        bh_train_l.append(((stocknum*(0.998*dfo['adj_close_price'][len(dfo)-1])+(100000-stocknum*dfo['adj_close_price'][0]*1.002))-100000)/1000)
        test_l.append((ret-100000)/1000)
        print("B&H_train :",bh_train_l)
        print("ats_train :",train_l) 
        print("B&H_test :",bh_test_l)
        print ("ATS test:",b,(ret-100000)/1000)
        
    print("B&H_train :",bh_train_l)
    print("ats_train :",train_l) 
    print("B&H_test :",bh_test_l)
    print("ats_test :",test_l)

In [ ]:
fobj()

In [11]:
fobj()

[array([0.11388528, 0.06667182, 0.02407692, 0.09028309, 0.0326584 ,
       0.13917478, 0.00534666, 0.1036111 , 0.41096295, 0.013329  ,
       0.21640011]), 69.76992641155245, 0]
B&H_train : [22.50024920012419]
ats_train : [69.76992641155245]
B&H_test : [7.131995330033067]
ATS test: 0 -0.3061604301147454
[array([0.01332371, 0.06784081, 0.0424797 , 0.0876485 , 0.04582086,
       0.37668988, 0.02249422, 0.12860996, 0.00841778, 0.20667458,
       0.27266861]), 38.09099657524109, 1]
B&H_train : [22.50024920012419, 2.143137023322226]
ats_train : [69.76992641155245, 38.09099657524109]
B&H_test : [7.131995330033067, -27.927248053864272]
ATS test: 1 0.0
[array([0.01842561, 0.16484154, 0.02824634, 0.05979428, 0.08553423,
       0.0750237 , 0.00572295, 0.15884859, 0.37261096, 0.0309518 ,
       0.21819463]), 136.48010346142584, 2]
B&H_train : [22.50024920012419, 2.143137023322226, -24.509857640687244]
ats_train : [69.76992641155245, 38.09099657524109, 136.48010346142584]
B&H_test : [7.13199533003

In [12]:
fobj()

[array([0.00272065, 0.01741899, 0.08864687, 0.1593415 , 0.18278696,
       0.03850749, 0.00506377, 0.17880208, 0.24685829, 0.0798534 ,
       0.1832212 ]), 77.74592373424532, 0]
B&H_train : [22.50024920012419]
ats_train : [77.74592373424532]
B&H_test : [7.131995330033067]
ATS test: 0 0.8049144543914445
[array([0.01135674, 0.003812  , 0.0598458 , 0.01502772, 0.11897253,
       0.61063595, 0.08655647, 0.00774445, 0.08157166, 0.00447668,
       0.1635238 ]), 43.01970428479768, 1]
B&H_train : [22.50024920012419, 2.143137023322226]
ats_train : [77.74592373424532, 43.01970428479768]
B&H_test : [7.131995330033067, -27.927248053864272]
ATS test: 1 -15.606325532012939
[array([0.02202119, 0.00608634, 0.08980691, 0.12055653, 0.03630556,
       0.00585731, 0.01216578, 0.18425732, 0.25993086, 0.2630122 ,
       0.25034826]), 108.50150200170138, 2]
B&H_train : [22.50024920012419, 2.143137023322226, -24.509857640687244]
ats_train : [77.74592373424532, 43.01970428479768, 108.50150200170138]
B&H_test :

In [13]:
fobj()

[array([0.01187328, 0.13933082, 0.0032994 , 0.01492205, 0.06320123,
       0.04331596, 0.01012869, 0.16191881, 0.04437797, 0.50763179,
       0.1712499 ]), 52.73055627883918, 0]
B&H_train : [22.50024920012419]
ats_train : [52.73055627883918]
B&H_test : [7.131995330033067]
ATS test: 0 2.5304493645629846
[array([0.01253428, 0.08082094, 0.02328014, 0.03444144, 0.08325547,
       0.01800289, 0.0660436 , 0.01251445, 0.4633268 , 0.20577999,
       0.19176243]), 41.31024797445675, 1]
B&H_train : [22.50024920012419, 2.143137023322226]
ats_train : [52.73055627883918, 41.31024797445675]
B&H_test : [7.131995330033067, -27.927248053864272]
ATS test: 1 -11.241998997703558
[array([0.18927284, 0.01685913, 0.02412433, 0.03218865, 0.05052385,
       0.01687397, 0.03280868, 0.01942607, 0.16006969, 0.45785278,
       0.26963447]), 84.24779022906878, 2]
B&H_train : [22.50024920012419, 2.143137023322226, -24.509857640687244]
ats_train : [52.73055627883918, 41.31024797445675, 84.24779022906878]
B&H_test : [

In [14]:
fobj()

[array([0.08774115, 0.31928888, 0.02975375, 0.05417198, 0.0737828 ,
       0.09848487, 0.01391226, 0.17159107, 0.12591811, 0.02535514,
       0.29132439]), 38.80184141511534, 0]
B&H_train : [22.50024920012419]
ats_train : [38.80184141511534]
B&H_test : [7.131995330033067]
ATS test: 0 5.476079811645497
[array([0.02286984, 0.12048519, 0.02111657, 0.01111272, 0.01678441,
       0.0725417 , 0.0799517 , 0.13089345, 0.51079628, 0.01344814,
       0.24468175]), 26.16600570013431, 1]
B&H_train : [22.50024920012419, 2.143137023322226]
ats_train : [38.80184141511534, 26.16600570013431]
B&H_test : [7.131995330033067, -27.927248053864272]
ATS test: 1 -37.17070466848755
[array([0.08230848, 0.07629678, 0.00256619, 0.05364761, 0.03299746,
       0.092647  , 0.10172867, 0.00696141, 0.36958615, 0.18126026,
       0.10088758]), 145.7147995066262, 2]
B&H_train : [22.50024920012419, 2.143137023322226, -24.509857640687244]
ats_train : [38.80184141511534, 26.16600570013431, 145.7147995066262]
B&H_test : [7.

In [ ]:
#test
import pandas as pd
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('AAPL_2020_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['sma89_signal']
    sl2=dfo['obv_macd_signal']
    sl3=dfo['cdc_signal']
    sl4=dfo['rsi_signal']
    sl5=dfo['renko_signal']
    sl6=dfo['sar_signal']
    sl7=dfo['macd_signal']
    sl8=dfo['stoch_signal']
    sl9=dfo['obv_renko_signal']
    sl10=dfo['aroon_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)

In [ ]:
(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])/dfo['adj_close_price'][0]*100

In [150]:
#test
import pandas as pd
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('TSLA_18_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['sma89_signal']
    sl2=dfo['obv_macd_signal']
    sl3=dfo['cdc_signal']
    sl4=dfo['rsi_signal']
    sl5=dfo['renko_signal']
    sl6=dfo['sar_signal']
    sl7=dfo['macd_signal']
    sl8=dfo['stoch_signal']
    sl9=dfo['obv_renko_signal']
    sl10=dfo['aroon_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)

-3.8563480945091433

In [149]:
(dfo['adj_close_price'][len(dfo)-1]-dfo['adj_close_price'][0])/dfo['adj_close_price'][0]*100

4.161849649068813

In [151]:
listna

[0.05171901078056007,
 0.09168125927799935,
 0.0635959532800144,
 0.19770249244774873,
 0.06882487224742326,
 0.12934489843397567,
 0.07462249921901046,
 0.14327630557640067,
 0.14077705830467716,
 0.1346100853889713,
 0.1493072239607629]

In [63]:
listna

[0.03418788291218656,
 0.03149076244234548,
 0.15037019569028254,
 0.1325433769862197,
 0.14793535036282862,
 0.08853880692902165,
 0.09635943910162295,
 0.12403901528853958,
 0.12794771591086848,
 0.15992870893847594,
 0.19253667530824434]

In [ ]:
#BH 16 107.4283664979756
#back 16 110.85032925866324
#BH 4.161849649068813
#Forward 0

In [16]:
listna=[0.07209369806093725,
 0.07185038480865677,
 0.07605418723127022,
 0.046688922200022735,
 0.05800967858322942,
 0.20641780081608485,
 0.13221615163816813,
 0.17658586648082206,
 0.13257461186240724,
 0.12475782848656121,
 0.23315362570785392]

In [170]:
a=[0.07209369806093725,
 0.07185038480865677,
 0.07605418723127022,
 0.046688922200022735,
 0.05800967858322942,
 0.20641780081608485,
 0.13221615163816813,
 0.17658586648082206,
 0.13257461186240724,
 0.12475782848656121,
 0.23315362570785392]
td=a[10]
w=a[0:10]
df=splittest[0]

decision_d_l=[]
il=[]
global count1,count2,count3,sell_idx,buy_idx,sl1
mon=100000
amount=0
sumall=0
df=df.reset_index()
sl1=df['sma89_signal']
sl2=df['obv_macd_signal']
sl3=df['cdc_signal']
sl4=df['rsi_signal']
sl5=df['renko_signal']
sl6=df['sar_signal']
sl7=df['macd_signal']
sl8=df['stoch_signal']
sl9=df['obv_renko_signal']
sl10=df['aroon_signal']

for i in range(0,len(df)):
    decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
    if i!=0:
        if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.8:
                mon+=amount*df['Close'][i]*0.998
                amount=0
                sell_idx.append(i)
    if decision_d>td:
        sumall=mon//df['adj_close_price'][i]
        amount+=sumall
        mon-=sumall*df['adj_close_price'][i]*1.002
        count1+=1
        buy_idx.append(i)
    elif decision_d<-td:
        mon+=amount*df['adj_close_price'][i]*0.998
        amount=0
        count2+=1
        sell_idx.append(i)
    else:
        count3+=1
decision_d_l.append(mon+amount*df['adj_close_price'][i])
il.append(i)
ret=mon+amount*df['adj_close_price'][len(df)-1]
print (("TEST",b,ret-100000)/1000)

NameError: name 'b' is not defined

In [168]:
sl1

155    1
156    1
157    1
158   -1
159   -1
160    1
161    1
162    1
163    1
164    1
165   -1
166   -1
167   -1
168   -1
169   -1
170   -1
171   -1
172   -1
173   -1
174   -1
175   -1
176   -1
177   -1
178   -1
179   -1
180   -1
181   -1
182   -1
183   -1
184   -1
185   -1
Name: sma89_signal, dtype: int64

In [ ]:
aapl [array([0.00651714, 0.0418374 , 0.00165859, 0.29855315, 0.03284619,
       0.00136117, 0.00648658, 0.32601712, 0.06353603, 0.22118663,
       0.2675562 ]), 109.48773398520277, 0]
training 59.43858069024412
test 35.39426686111843
TEST 0 0.404597674423203
[array([0.01142926, 0.02151963, 0.00302768, 0.33327822, 0.05738381,
       0.01156214, 0.10782697, 0.23002569, 0.06365855, 0.16028805,
       0.21954648]), 191.48735877567296, 1]
training 103.73195749151867
test -14.437495640525492
TEST 1 2.2892019599533233
[array([0.14225291, 0.01633075, 0.02125917, 0.14837138, 0.00860034,
       0.23607799, 0.01275173, 0.32604496, 0.06683945, 0.02147131,
       0.16668405]), 213.49449069986724, 2]
training 81.63249449970678
test 53.50086231676958
TEST 2 46.38210911050415
[array([0.0642284 , 0.36078571, 0.20965265, 0.05398472, 0.0071029 ,
       0.02499766, 0.00446481, 0.02171799, 0.03221287, 0.22085228,
       0.24541019]), 243.93027599285892, 3]
training 186.98489691064046
test 0.0
TEST 3 0.0

In [ ]:
xom [array([0.05653171, 0.04158094, 0.00966091, 0.14019216, 0.0191391 ,
       0.26082285, 0.00551169, 0.37116798, 0.08228806, 0.01310459,
       0.2248158 ]), 19.225797561813366, 0]
training -5.170508806093523
test -2.7033271208700316
TEST 0 0.0
[array([0.00311368, 0.03030103, 0.33966217, 0.24737007, 0.02273162,
       0.02808046, 0.03276149, 0.00477225, 0.27906039, 0.01214684,
       0.18180397]), 13.179899188735988, 1]
training 7.2692574914808645
test 14.48207968124395
TEST 1 20.022729757904017
[array([0.12217482, 0.01021499, 0.03710938, 0.35005955, 0.08345536,
       0.00414276, 0.03887741, 0.12135565, 0.02647102, 0.20613907,
       0.24844787]), 17.902870460334757, 2]
training 13.253616245776357
test -7.204463019827541
TEST 2 -0.05212197679138626
[array([0.09972493, 0.09580023, 0.124599  , 0.03161699, 0.42068358,
       0.08300646, 0.02856566, 0.00160072, 0.0215717 , 0.09283073,
       0.10452025]), 17.789281327056816, 3]
training -0.9895668547774112
test -6.217451020276567
TEST 3 11.146635102462765

In [ ]:
aapl [array([0.11344563, 0.00300135, 0.0460275 , 0.32719982, 0.05262101,
       0.15073093, 0.13365692, 0.01021486, 0.06260673, 0.10049526,
       0.14119277]), 74.86024047516632, 0]
B&H_train : [37.314737311231]
ats_train : [74.86024047516632]
B&H_test : [20.727068690554734]
ATS test: 0 7.382185850845315
[array([9.36034461e-02, 5.79979771e-02, 4.93700458e-01, 3.73656089e-02,
       6.18699956e-03, 8.60879761e-03, 4.30137090e-04, 6.50439325e-03,
       8.95809159e-02, 2.06021267e-01, 1.95738293e-01]), 76.07139998421856, 1]
B&H_train : [37.314737311231, 47.25351397348932]
ats_train : [74.86024047516632, 76.07139998421856]
B&H_test : [20.727068690554734, 52.550442252856534]
ATS test: 1 13.7726287834549
[array([0.08451361, 0.02315076, 0.04054459, 0.10996198, 0.0918642 ,
       0.07582466, 0.00929327, 0.53884508, 0.0068166 , 0.01918526,
       0.15783876]), 144.99826585281372, 2]
B&H_train : [37.314737311231, 47.25351397348932, 110.91450426389454]
ats_train : [74.86024047516632, 76.07139998421856, 144.99826585281372]
B&H_test : [20.727068690554734, 52.550442252856534, -31.435810916914722]
ATS test: 2 28.80950232375336
[array([0.03024085, 0.00377284, 0.01964859, 0.14752845, 0.19188683,
       0.27554433, 0.00294231, 0.16322036, 0.14006471, 0.02515074,
       0.15824992]), 163.2337241676788, 3]
B&H_train : [37.314737311231, 47.25351397348932, 110.91450426389454, 87.21042046741259]
ats_train : [74.86024047516632, 76.07139998421856, 144.99826585281372, 163.2337241676788]
B&H_test : [20.727068690554734, 52.550442252856534, -31.435810916914722, 99.07721291849401]
ATS test: 3 50.18412416074373
B&H_train : [37.314737311231, 47.25351397348932, 110.91450426389454, 87.21042046741259]
ats_train : [74.86024047516632, 76.07139998421856, 144.99826585281372, 163.2337241676788]
B&H_test : [20.727068690554734, 52.550442252856534, -31.435810916914722, 99.07721291849401]
ats_test : [7.382185850845315, 13.7726287834549, 28.80950232375336, 50.18412416074373]